In [1]:
# %run ./__init__edwh__notebooks.ipynb
# uit te voeren via `jupyter nbconvert --to html --TemplateExporter.exclude_input=True  Leids\ aanbod\ statistieken.ipynb` via een console in ~/work 
# dan krijg je een mooie HTML met alleen de uitvoer. 

In [2]:
!pip install tabulate > /dev/null 

In [8]:
import os, datetime 
from dotenv import load_dotenv
from tabulate import tabulate
from nocodb import NocoClient 
load_dotenv('.env')
noco = NocoClient(
    "https://nocodb.edwh.nl",
    "mike",
    os.getenv('NOCODB_MIKE_KEY'),
)
print("Rapport gegenereerd op",datetime.datetime.now())

Rapport gegenereerd op 2022-05-05 13:56:52.628450


<!--
# rapportage voor Alette 
[x] Hoeveel aanbod in totaal, KO, PO, VO, HO, MBO  
[x] Hoeveel aanbod per aanbieder   
[x] Hoeveel aanbod per menu kaart item (quickfilter tags), ook per KO,PO,VO,Ho,MBO  
[x] Hoeveel van het aanbod gaat over 1 of meerdere activiteiten (zowel het aantal posts, en hoeveel dagen per activiteit) - alleen totaal  
[x] draft posts negeren 
-->

In [4]:
from collections import Counter 
from typing import List 
from attr import define, field, asdict 
import httpx 

@define
class Sector:
    noco:NocoClient
    where:tuple 
    aanbod:List[dict] = field(init=False, default=lambda:[])
    # size:int = field(init=False, default=-1)
    
    def __attrs_post_init__(self):
        # self.aanbod = noco.aanbod.list(dict(where=self.where, limit=999999, fields='Id,titel,slug,aanbieders,tags'))['list']
        select = '?select='+self.where[2] if self.where else ''
        url = 'https://n8n.edwh.nl/webhook/b6e3e155-39e2-424c-84bb-979125ab9e3a'+select
        self.aanbod = httpx.get(url).json()['list']

    def __len__(self):
        return len(self.aanbod)
    
    def aanbieders(self):
        aanbieders = []
        for aanbod in self.aanbod: 
            aanbieders.extend(aanbod['aanbieders'])
        return {aanbieder['id']:aanbieder for aanbieder in aanbieders}
    
    def aanbod_count_per_aanbieder(self):
        items_per_aanbieder = Counter()
        for aanbod in self.aanbod:
            for aanbieder in aanbod['aanbieders']:
                items_per_aanbieder[aanbieder['name']] += 1
        return items_per_aanbieder
    
    def aanbod_per_aanbieder(self):
        items_per_aanbieder = {}
        for aanbod in self.aanbod:
            for aanbieder in aanbod['aanbieders']:
                items_per_aanbieder[aanbieder['name']].append(aanbod)
        return items_per_aanbieder
    
    def aanbod_per_tag(self, tag_slug):
        return len([a for a in self.aanbod if tag_slug in [s['slug'] for s in a['tags']]])

    

In [5]:
sector_totaal = Sector(noco=noco, where=None)
sector_ko = Sector(noco=noco, where=('tags','like','hash-ko'))
sector_po = Sector(noco=noco, where=('tags','like','hash-po'))
sector_vo = Sector(noco=noco, where=('tags','like','hash-vo'))
sector_mbo = Sector(noco=noco, where=('tags','like','hash-mbo'))
sector_ho = Sector(noco=noco, where=('tags','like','hash-ho'))

themas = ['onderwijsondersteuning', 'leerlingondersteuning','gezond-en-bewegen','taalontwikkeling','mentaal-welzijn','sociale-veiligheid','aanvullend-en-verrijkend-onderwijs','professionalisering','schoolgebouwen-en-omgeving']
headers=['Sector','aanbod','aanbieders']
table=[
    ['Totaal',len(sector_totaal), len(sector_totaal.aanbieders())], 
    ['KO',len(sector_ko), len(sector_ko.aanbieders())], 
    ['PO',len(sector_po), len(sector_po.aanbieders())], 
    ['VO',len(sector_vo), len(sector_vo.aanbieders())], 
    ['MBO',len(sector_mbo), len(sector_mbo.aanbieders())], 
    ['HO',len(sector_ho), len(sector_ho.aanbieders())], 
]
for tag in themas:
    headers.append(tag)
    for row in table: 
        sector_naam = row[0]
        sector = locals()[f'sector_{sector_naam.lower()}']
        row.append(sector.aanbod_per_tag(tag))
tabulate(table,headers, tablefmt='html')

Sector,aanbod,aanbieders,onderwijsondersteuning,leerlingondersteuning,gezond-en-bewegen,taalontwikkeling,mentaal-welzijn,sociale-veiligheid,aanvullend-en-verrijkend-onderwijs,professionalisering,schoolgebouwen-en-omgeving
Totaal,132,33,13,16,11,12,18,9,21,25,1
KO,26,10,1,5,1,6,3,3,1,10,0
PO,84,30,12,14,8,10,14,7,13,17,1
VO,78,24,6,4,8,2,12,5,13,16,0
MBO,1,1,0,0,0,0,0,0,0,0,0
HO,1,1,0,0,0,0,0,0,0,1,0


In [6]:
print('Aanbod per aanbieder')
for name, count in sector_totaal.aanbod_count_per_aanbieder().most_common():
    print(f'{name:>50}: {count}')

Aanbod per aanbieder
               Onderwijsnetwerk Zuid-Holland (ONZ): 17
          De Leidse Aanpak voor Talentontwikkeling: 15
                                         Technolab: 13
                 CJG Leiden en GGD Hollands Midden: 11
      Centrum onderwijs en innovatie Leiden (COIL): 10
                                      JES Rijnland: 10
                         ICLON Universiteit Leiden: 9
               Kenniscentrum Angst & Stress Leiden: 8
                         Leiden Education Fieldlab: 6
                                            BplusC: 5
                                   Gemeente Leiden: 4
                              Cultuureducatiegroep: 3
                                 Hogeschool Leiden: 2
                               Universiteit Leiden: 2
                                       SOL netwerk: 2
         Gemeentelijke Schoolsportcommissie Leiden: 2
                                       Studio Moio: 2
                                    Stichting Move: 2
 

In [7]:
aanbod = sector_totaal.aanbod
table = []
import re
activiteit_slug_re = re.compile(r'hash-\d\d\d\d-\d\d-\d\d')
headers = ['Aanbod','is activiteit?','aantal dagen','sectoren', 'agenda']
for a in aanbod: 
    table.append([
        a['titel'],
        'activiteit' in [t['title'] for t in a['tags']],
        sum((1 for t in a['tags'] if activiteit_slug_re.match(t['slug']))), 
        ' '.join(t['title'] for t in a['tags'] if t['slug'].replace('hash-','') in ('kopombohovo')),
        ' '.join('#'+t['slug'] for t in a['tags'] if t['slug'] in themas)
    ])
table.sort(key=lambda row:row[2], reverse=True)
tabulate(table, headers, tablefmt='html')

Aanbod,is activiteit?,aantal dagen,sectoren,agenda
Leids Onderwijs Project (LOP),True,40,#PO,#leerlingondersteuning
Letterlijk Leiden,True,38,#KO #PO #VO,#aanvullend-en-verrijkend-onderwijs
Weekendklas,True,35,#PO,#aanvullend-en-verrijkend-onderwijs #leerlingondersteuning #mentaal-welzijn #onderwijsondersteuning
PRÉ HBO,True,26,#VO,#aanvullend-en-verrijkend-onderwijs
Opleiding Expert Slimme Peuters en Kleuters 2021-2022,True,11,#KO #PO,#professionalisering
Docenttraining Activerend leren in de praktijk,True,10,#VO,#professionalisering
Vakdidactief-café voor nieuwe docenten geschiedenis,True,8,#VO,
Vakdidactief-café MVT,True,7,#VO,
Seminars RITHA-opleiding,True,6,#KO #PO #VO,#professionalisering
Cursus sterrenkunde,True,5,#VO,
